<a href="https://colab.research.google.com/github/aruntakhur/GEU_ML_Jan_25/blob/main/Demo_CoT_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!git clone https://github.com/itsnamgyu/reasoning-teacher.git

Cloning into 'reasoning-teacher'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 168 (delta 56), reused 138 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (168/168), 19.19 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (56/56), done.


# New Section

In [3]:
%cd reasoning-teacher

/content/reasoning-teacher


In [4]:
!pip install -e .

Obtaining file:///content/reasoning-teacher
  Preparing metadata (setup.py) ... done
  Running setup.py develop for reasoning_teacher


In [8]:
!pip install datasets

In [13]:
from pathlib import Path

In [14]:

# ✅ Sample CoT-style training data (You can expand this)
import json
Path("data").mkdir(exist_ok=True)
sample_data = [
    {
        "input": "Q: If a train travels at 60 km/h for 2 hours, how far does it go?\nA: Let's think step by step.",
        "target": "The train travels at 60 km/h. In 2 hours, it will travel 60 × 2 = 120 km. Therefore, the answer is 120 km."
    },
    {
        "input": "Q: What is 15 + 27?\nA: Let's think step by step.",
        "target": "15 + 27 = 42. Therefore, the answer is 42."
    }
]
with open("data/finetune_cot_train.json", "w") as f:
    json.dump(sample_data, f)

In [15]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

model_name = "google/flan-t5-small"  # or t5-base, etc.
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Apply LoRA (PEFT)
peft_config = LoraConfig(
    r=8, lora_alpha=32, target_modules=["q", "v"], lora_dropout=0.05, bias="none"
)
model = get_peft_model(model, peft_config)

# Load reasoning dataset from repo
from datasets import load_json
train_ds = load_json("data/finetune_cot_train.json") #data/finetune_cot_train.json
# Tokenize
def tokenize(batch):
    inp = batch["input"]
    tgt = batch["target"]
    model_inputs = tokenizer(inp, truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(tgt, truncation=True, padding="max_length", max_length=512).input_ids
    model_inputs["labels"] = labels
    return model_inputs

train_ds = train_ds.map(tokenize, batched=True)

# Training setup
training_args = TrainingArguments(
    output_dir="cot_student",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=50,
    save_total_limit=2,
    fp16=True,  # if GPU supports it
)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_ds
)
trainer.train()


ImportError: cannot import name 'load_json' from 'datasets' (/usr/local/lib/python3.11/dist-packages/datasets/__init__.py)